Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Mohini Khedekar"
COLLABORATORS = ""

---

# Rosetta Energy Score Functions

A basic function of Rosetta is calculating the energy or score of a biomolecule. Rosetta has a standard energy function for all-atom calculations as well as several scoring functions for low-resolution protein representations. See https://www.ncbi.nlm.nih.gov/pubmed/28430426 for a review on the all-atom score functions.

You can also tailor an energy function by including scoring terms of your choice with custom weights.

**Chapter contributors:**

- Kathy Le (Johns Hopkins University) adapted this chapter from the [PyRosetta Workshops](https://www.amazon.com/PyRosetta-Interactive-Platform-Structure-Prediction-ebook/dp/B01N21DRY8) (J. J. Gray, S. Chaudhury, S. Lyskov, J. Labonte, 2017).

# Score Function Basics
Keywords: score function, ScoreFunction(), get_score_function(), set_weight(), show(), etable_atom_pair_energies(), Atom objects, get_hbonds(), nhbonds(), residue_hbonds()

In [1]:
import sys
if 'google.colab' in sys.modules:
  !pip install pyrosettacolabsetup
  import pyrosettacolabsetup
  pyrosettacolabsetup.mount_pyrosetta_install()
import pyrosetta
pyrosetta.init()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin/wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2023.19+release.d7aa7f94e8b-cp310-cp310-linux_x86_64.whl
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2023.19+release.d7aa7f94e8be5e9d5110d37f167c2a7afd30c530 2023-05-08T16:22:16] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r348 2023.19+release.d7aa7f94e8b d7aa7f94e8be5e9d5110

**Make sure you are in the directory with the pdb files:**



In [ ]:
# cd into the directory

In this module, we will explore the PyRosetta score function interface. You will learn to inspect energies of a biomolecule at the whole protein, per-residue, and per-atom level. Finally, you will gain practice applying the energies to answering biological questions involving proteins. For these exercises, we will use the protein Ras (PDB 6q21). You should have the PDB file "6Q21_A.pdb" in your `inputs/` directory. Load it into a Pose variable named `ras` with the pyrosetta.pose_from_pdb method. 

In [3]:
# YOUR CODE HERE
ras = pyrosetta.pose_from_pdb("/content/google_drive/MyDrive/codeschool2023/6q21.pdb")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.650683 seconds.
core.import_pose.import_pose: File '/content/google_drive/MyDrive/codeschool2023/6q21.pdb' automatically determined to be of type PDB
core.chemical.GlobalResidueTypeSet: Loading (but possibly not actually using) 'GCP' from the PDB components dictionary for residue type 'pdb_GCP'


To score a protein, you will begin by defining a score function using the `get_score_function(is_fullatom: bool)` method in the `pyrosetta.teaching` namespace. Specifying `True` will return the default `ref2015` all-atom energy function, while `False` will specify the default centroid score function.

Create a PyRosetta score function using:
```
sfxn = get_score_function(True)
```

In [4]:
from pyrosetta.teaching import *

sfxn = get_score_function(True)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: Database file opened: scoring/score_functions/rama/fd/

You can see the terms, weights, and energy method options by printing the score function:

```
print(sfxn)
```

In [5]:
# YOUR CODE HERE
print(sfxn)

ScoreFunction::show():
weights: (fa_atr 1) (fa_rep 0.55) (fa_sol 1) (fa_intra_rep 0.005) (fa_intra_sol_xover4 1) (lk_ball_wtd 1) (fa_elec 1) (pro_close 1.25) (hbond_sr_bb 1) (hbond_lr_bb 1) (hbond_bb_sc 1) (hbond_sc 1) (dslf_fa13 1.25) (omega 0.4) (fa_dun 0.7) (p_aa_pp 0.6) (yhh_planarity 0.625) (ref 1) (rama_prepro 0.45)
energy_method_options: EnergyMethodOptions::show: aa_composition_setup_files: 
EnergyMethodOptions::show: mhc_epitope_setup_files: 
EnergyMethodOptions::show: netcharge_setup_files: 
EnergyMethodOptions::show: aspartimide_penalty_value: 25
EnergyMethodOptions::show: etable_type: FA_STANDARD_DEFAULT
analytic_etable_evaluation: 1
EnergyMethodOptions::show: method_weights: ref 1.32468 3.25479 -2.14574 -2.72453 1.21829 0.79816 -0.30065 2.30374 -0.71458 1.66147 1.65735 -1.34026 -1.64321 -1.45095 -0.09474 -0.28969 1.15175 2.64269 2.26099 0.58223
EnergyMethodOptions::show: method_weights: free_res
EnergyMethodOptions::show: unfolded_energies_type: UNFOLDED_SCORE12
EnergyMeth

## Practice: List the terms in the energy function and their relative weights

**Hint:** look at the top line that starts with 'weights'

In [ ]:
## Your Response Here

#(fa_atr 1) (fa_rep 0.55) (fa_sol 1) (fa_intra_rep 0.005) (fa_intra_sol_xover4 1) (lk_ball_wtd 1) (fa_elec 1) (pro_close 1.25) (hbond_sr_bb 1) (hbond_lr_bb 1) (hbond_bb_sc 1) (hbond_sc 1) (dslf_fa13 1.25) (omega 0.4) (fa_dun 0.7) (p_aa_pp 0.6) (yhh_planarity 0.625) (ref 1) (rama_prepro 0.45)

## Custom energy functions

You can also create a custom energy function that includes select terms. Typically, creating a whole new score function is unneccesary because the current one works well in most cases. However, tweaking the current enrgy function by reassigning weights and adding certain energy terms can be useful.

Here, we will make an example energy function with only the van der Waals attractive and repulsive terms, both with weights of 1. We need to use the `set_weight()`. Make a new `ScoreFunction` and set the weights accordingly. This is how we set the full-atom attractive (`fa_atr`) and the full-atom repulsive (`fa_rep`) terms.

```
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)
```

In [6]:
sfxn2 = ScoreFunction()
sfxn2.set_weight(fa_atr, 1.0)
sfxn2.set_weight(fa_rep, 1.0)

Lets compare the score of `ras` using the full-atom `ScoreFunction` versus the `ScoreFunction` we made above using only the attractive and repulsive terms.

First, print the total energy of `ras` using `print(sfxn(ras))`
Then, print the attractive and repulsive energy only of `ras` using `print(sfxn2(ras))`

In [7]:
# print total energy of ras
print(sfxn(ras))

# print the attractive and repulsive energy of ras
print(sfxn2(ras))

basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.25675 seconds to load from binary
3856.127931662087
65.88066458854428


## Energy Breakdown

Using the full-atom `ScoreFunction` `sfxn`, break the energy of `ras` down into its individual pieces with the `sfxn.show(ras)` method. Which are the three most dominant contributions, and what are their values? Is this what you would have expected? Why? Note which terms are positive and negative

In [8]:
# use the sfxn.show() method
sfxn.show(ras)

core.scoring.ScoreFunction: 
------------------------------------------------------------
 Scores                       Weight   Raw Score Wghtd.Score
------------------------------------------------------------
 fa_atr                       1.000   -4465.653   -4465.653
 fa_rep                       0.550    4531.534    2492.343
 fa_sol                       1.000    3153.321    3153.321
 fa_intra_rep                 0.005    3064.638      15.323
 fa_intra_sol_xover4          1.000     200.904     200.904
 lk_ball_wtd                  1.000     -54.711     -54.711
 fa_elec                      1.000   -1083.192   -1083.192
 pro_close                    1.250     270.565     338.206
 hbond_sr_bb                  1.000    -181.224    -181.224
 hbond_lr_bb                  1.000    -130.586    -130.586
 hbond_bb_sc                  1.000     -73.431     -73.431
 hbond_sc                     1.000     -53.769     -53.769
 dslf_fa13                    1.250       0.000       0.000
 omega  

In [ ]:
# Your response here: what are the three most dominant contributions?
#The 3 most dominant contributions are from energy functions fa_atr (-4465.653), fa_dun(3245.013), fa_sol(3153.321) since they have the highest corresponding magnitude for weighted score. 
#This means they affect the protein structure/the phenomenon of protein folding the most. 
#fa_atr has a negative weight meaning it is an attractive force which makes sense.
#this was expected since attractive forces and solvent forces have a huge impact on protein folding. 

Unweighted, individual component energies of each residue in a structure are stored in the `Pose` object and can be accessed by the `energies()` method. For example, to break down the energy into each residue's contribution, we use: 
```
print(ras.energies().show(<n>))
```
Where `<n>` is the residue number.

What are the total van der Waals, solvation, and hydrogen-bonding contributions of residue 24?

Note: The _backbone_ hydrogen-bonding terms for each residue are not available from the `Energies` object. You can get them by using EnergyMethodOptions. See http://www.pyrosetta.org/documentation#TOC-Hydrogen-Bonds-and-Hydrogen-Bond-Scoring.

In [10]:
print(ras.energies().show(24))

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)  24         -7.40         19.03          2.94          8.76          0.09         -0.11         -0.56          0.00          0.00          0.00          0.00          0.00          0.00          0.12          2.68          0.06          0.00          2.30          3.58
None


In [ ]:
#total van der waals energy = fa_rep - fa_atr = 0.55-1 = -0.45
#solvation energy = fa_sol= 2.94 kcal/mol
#hydrogen-bonding energy = hbond_bb_sc = 0.0
#the reason why hydrogen bonding energy is 0 is that they are being cancelled out by other the hydrogen bond energies from surroundign residues 

The van der Waals, solvation, and electrostatic terms are atom-atom pairwise energies calculated from a pre-tabulated lookup table, dependent upon the distance between the two atoms and their types. You can access this lookup table, called the `etable` directly to check these energy calculations on an atom-by-atom basis. Use the `etable_atom_pair_energies` function which returns a triplet of energies for attractive, repulsive and solvation scores.

(Note that the `etable_atom_pair_energies()` function requires `Residue` objects, and atom indices (integers), and not the `AtomID` objects we saw in Workshop #2. For more info, look at the [documentation](https://graylab.jhu.edu/PyRosetta.documentation/pyrosetta.toolbox.atom_pair_energy.html?highlight=etable_atom_pair_energies#pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies).)

**Practice:** What are the attractive, repulsive, solvation, and electrostatic components between the nitrogen of residue 24 and the oxygen of residue 20? 


In [18]:

res24 = ras.residue(24)
res20 = ras.residue(20)
res24_N = ras.residue(24).atom_index("N")
res20_O = ras.residue(20).atom_index("O")
pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies(res24, res24_N, res20, res20_O, sfxn)

(-0.1505855046001568, 0.0, 0.5903452111877214, 2.173111777247698)

attractive = -0.1505855046001568, repulsive = 0.0, solvation = 0.5903452111877214, electrostatic = 2.173111777247698

# Practice: Analyzing energy between residues
Keywords: pose_from_rcsb(), pdb2pose(), EMapVector()

In [20]:
# From previous section:
sfxn = get_score_function(True)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015


Analyze the energy between residues Y102 and Q408 in cetuximab (PDB code 1YY9, use the `pyrosetta.toolbox.pose_from_rcsb` function to download it and load it into a new `Pose` object) by following the steps below. 

A. Internally, a Pose object has a list of residues, numbered starting from 1. To find the residue numbers of Y102 of chain D and Q408 of chain A, use the residue chain identifier and the PDB residue number to convert to the pose numbering using the `pose2pdb()` method:

```
pose = pyrosetta.toolbox.pose_from_rcsb("1YY9")
res102 = pose.pdb_info().pdb2pose("D", 102)
res408 = pose.pdb_info().pdb2pose("A", 408)
```

In [ ]:
# get the pose numbers for Y102 (chain D) and Q408 (chain A)

In [21]:
pose = pyrosetta.toolbox.pose_from_rcsb("1YY9")
res102 = pose.pdb_info().pdb2pose("D", 102)
res408 = pose.pdb_info().pdb2pose("A", 408)

core.import_pose.import_pose: File '1YY9.clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 2
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue LYS 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on r

B. Score the pose and determine the van der Waals energies and solvation energy between these two residues. Use the following commands to isolate contributions from particular pairs of residues, where `rsd102` and `rsd408` are the two residue objects of interest from above (not the residue number -- use `pose.residue(res_num)` to access the objects): 

```
emap = EMapVector()
sfxn.eval_ci_2b(pose.residue(res102), pose.residue(res408), pose, emap)
print(emap[fa_atr])
print(emap[fa_rep])
print(emap[fa_sol])
```

In [24]:
emap = EMapVector()
sfxn.eval_ci_2b(pose.residue(res102), pose.residue(res408), pose, emap)
print(emap[fa_atr])
print(emap[fa_rep])
print(emap[fa_sol])

-1.2098840439684349
0.10835353848860374
1.5729435146961963


What is the energy of Proline34? What about Alanine66? Which residue has lower energy?

In [26]:
#energy for proline34
print(ras.energies().show(34))

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)  34         -2.69          0.77          2.19          0.29          0.05         -0.07          0.09         28.89          0.00          0.00          0.00          0.00          0.00          0.70         13.95         -0.16          0.00         -1.64          0.86
None


In [27]:
#energy for alanine66
print(ras.energies().show(66))

core.scoring.Energies: E               fa_atr        fa_rep        fa_sol  fa_intra_repfa_intra_sol_x   lk_ball_wtd       fa_elec     pro_close   hbond_sr_bb   hbond_lr_bb   hbond_bb_sc      hbond_sc     dslf_fa13         omega        fa_dun       p_aa_pp yhh_planarity           ref   rama_prepro
core.scoring.Energies: E(i)  66         -1.99          0.82          1.68          0.64          0.00          0.04          2.18          0.00          0.00          0.00          0.00          0.00          0.00          0.37          0.00          0.48          0.00          1.32          5.28
None


fa_atr energy is higher for proline34, but fa_rep is higher for alanine, fa_sol is higher for proline34, but fa_elec is higher for alanine66. However, all the different kinds of hydrogen bond energies are 0 for both of them. 

In [28]:
#total energy for proline34
ras.energies().residue_total_energy(34)

44.79722328768828

In [29]:
#total energy for alanine66
ras.energies().residue_total_energy(66)

6.493099323481994

In [ ]:
# your response here
proline34 has a higher energy level than alanine66. 

For residue 24 from ras, it corresponds to isoleucine. This has a alkyl group as a side chain (non-polar) and hence there are no electronegative atoms that are bonded with hydrogen atoms in the side chain for hydrogen bonding to take place. Hence, mostly hydrophobic interactions would take place. 

## References
This Jupyter notebook is an adapted version of "Workshop #3: Scoring" in the PyRosetta workbook: https://graylab.jhu.edu/pyrosetta/downloads/documentation/pyrosetta4_online_format/PyRosetta4_Workshop3_Scoring.pdf